In [1]:
#  必要な素材の収集
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

# ランダムシードの作成
np.random.seed(20160703)
tf.set_random_seed(20160703)

# その他コンフィグ（いつもの）
%config IPCompleter.greedy=True
%matplotlib inline
%matplotlib notebook

/Users/yamamotomasaomi/.pyenv/versions/anaconda3-5.1.0/envs/makuwo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# mnistデータの収集
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# フィルターに対応するvariableを用意して、入力データに、フィルターと
# プーリング層（画像の画質を落とすことによって結果の質をあげることができるための処理を行う層）を作る。

# フィルターの作成
num_filters = 16

# Placeholderの設定
x = tf.placeholder(tf.float32, [None, 784])

#プーリング層の設定？
x_image = tf.reshape(x, [-1, 28, 28, 1])

w_conv = tf.Variable(tf.truncated_normal([5, 5, 1,num_filters], 
                                         stddev = 0.1))
h_conv = tf.nn.conv2d(x_image, w_conv, 
                      strides=[1, 1, 1, 1], padding='SAME')

h_pool = tf.nn.max_pool(h_conv ,ksize = [1, 2, 2, 1], 
                        strides=[1, 2, 2, 1], padding='SAME')


In [4]:
# プーリング層からの出力を全結合層を経由してソフトマックス関数に入力する計算式を定義する
h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters])

# ノードの作成
num_units1 = 14*14*num_filters
num_units2 = 1024

# 隠れ層２の作成
w2 = tf.Variable(tf.truncated_normal([num_units1,num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
# 高速化のためか、reluを使用
hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2)

# 出力層の作成
w0 = tf.Variable(tf.zeros([num_units2,10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2,w0) + b0)

In [5]:
t = tf.placeholder(tf.float32, [None ,10])
loss = -tf.reduce_sum(t*tf.log(p))
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p ,1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
# Variableの初期化
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [18]:
# 演算開始
i = 0

for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts})
    
    if i % 100 == 0:
        loss_val , acc_val = sess.run([loss, accuracy],
            feed_dict={x:batch_xs, t:batch_ts})
        print ('Step: %d Loss:%f Accuracy: %f' % (i, loss_val, acc_val))
        saver.save(sess, '/Users/yamamotomasaomi/jupyter_Study/mdc_session', global_step=i)

Step: 100 Loss:5.819578 Accuracy: 0.990000
Step: 200 Loss:4.502027 Accuracy: 0.980000
Step: 300 Loss:2.877551 Accuracy: 0.990000
Step: 400 Loss:5.921979 Accuracy: 0.970000
Step: 500 Loss:3.490588 Accuracy: 1.000000
Step: 600 Loss:3.985793 Accuracy: 0.990000
Step: 700 Loss:8.472003 Accuracy: 0.980000
Step: 800 Loss:0.980080 Accuracy: 1.000000
Step: 900 Loss:4.753296 Accuracy: 0.980000
Step: 1000 Loss:2.811954 Accuracy: 0.990000
Step: 1100 Loss:6.041953 Accuracy: 0.990000
Step: 1200 Loss:1.026057 Accuracy: 1.000000
Step: 1300 Loss:0.572853 Accuracy: 1.000000
Step: 1400 Loss:5.186653 Accuracy: 0.990000
Step: 1500 Loss:0.345272 Accuracy: 1.000000
Step: 1600 Loss:0.665350 Accuracy: 1.000000
Step: 1700 Loss:2.350949 Accuracy: 0.990000
Step: 1800 Loss:2.550058 Accuracy: 0.990000
Step: 1900 Loss:1.068620 Accuracy: 1.000000
Step: 2000 Loss:0.907918 Accuracy: 1.000000
Step: 2100 Loss:0.189470 Accuracy: 1.000000
Step: 2200 Loss:0.400936 Accuracy: 1.000000
Step: 2300 Loss:5.978219 Accuracy: 0.9900

In [22]:
! ls mdc_session*

mdc_session                          mdc_session-3800.index
mdc_session-3600.data-00000-of-00001 mdc_session-3800.meta
mdc_session-3600.index               mdc_session-3900.data-00000-of-00001
mdc_session-3600.meta                mdc_session-3900.index
mdc_session-3700.data-00000-of-00001 mdc_session-3900.meta
mdc_session-3700.index               mdc_session-4000.data-00000-of-00001
mdc_session-3700.meta                mdc_session-4000.index
mdc_session-3800.data-00000-of-00001 mdc_session-4000.meta
